In [1]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.parallel
import torch.optim as optim
import torch.utils.data
from torch.autograd import variable

In [2]:
movies=pd.read_csv(r'C:/Users/Armaan/OneDrive/Desktop/Workspace/Machine_Learning/Datasets/movieRecommendation/movies.dat', sep='::',header=None,engine="python",encoding='latin-1')
users=pd.read_csv(r'C:/Users/Armaan/OneDrive/Desktop/Workspace/Machine_Learning/Datasets/movieRecommendation/users.dat', sep='::',header=None,engine="python",encoding='latin-1')
ratings=pd.read_csv(r'C:/Users/Armaan/OneDrive/Desktop/Workspace/Machine_Learning/Datasets/movieRecommendation/ratings.dat', sep='::',header=None,engine="python",encoding='latin-1')

In [3]:
movies.tail()

,0,1,2
3878,3948,Meet the Parents (2000),Comedy
3879,3949,Requiem for a Dream (2000),Drama
3880,3950,Tigerland (2000),Drama
3881,3951,Two Family House (2000),Drama
3882,3952,"Contender, The (2000)",Drama|Thriller


In [4]:
users.tail()

,0,1,2,3,4
6035,6036,F,25,15,32603
6036,6037,F,45,1,76006
6037,6038,F,56,1,14706
6038,6039,F,45,0,01060
6039,6040,M,25,6,11106


In [5]:
ratings

,0,1,2,3
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291
...,...,...,...,...
1000204,6040,1091,1,956716541
1000205,6040,1094,5,956704887
1000206,6040,562,5,956704746
1000207,6040,1096,4,956715648


In [52]:
training_set=pd.read_csv(r'C:/Users/Armaan/OneDrive/Desktop/Workspace/Machine_Learning/Datasets/movieRecommendation/ml-100k/u1.base',delimiter="\t")
test_set=pd.read_csv(r'C:/Users/Armaan/OneDrive/Desktop/Workspace/Machine_Learning/Datasets/movieRecommendation/ml-100k/u1.test',delimiter="\t")

In [53]:
training_set=np.array(training_set,dtype="int")
test_set=np.array(test_set,dtype="int")

In [54]:
n_users=int(max(training_set[:,0].max(),test_set[:,0].max()))
n_movies=int(max(training_set[:,1].max(),test_set[:,1].max()))
n_users,n_movies

(943, 1682)

In [57]:
training_set=torch.FloatTensor(training_set)
test_set=torch.FloatTensor(test_set)

In [58]:
training_set.shape

torch.Size([943, 1682])

In [59]:
class SAE(nn.Module): #stacked auto encoders
    def __init__(self,):
        super(SAE,self).__init__()
        nb_movies=1682
        self.fc1=nn.Linear(nb_movies,20)    # full connection 1
        self.fc2=nn.Linear(20,10)
        self.fc3=nn.Linear(10,20)
        self.fc4=nn.Linear(20,nb_movies)
        self.activation=nn.Sigmoid()
    def forward(self,x):
         x=self.activation(self.fc1(x))
         x=self.activation(self.fc2(x))
         x=self.activation(self.fc3(x))
         x=self.fc4(x)
         return x        

In [60]:
sae=SAE()
criterion=nn.MSELoss()
optimizer=optim.RMSprop(sae.parameters(),lr=0.01,weight_decay=0.5)

In [78]:
epochs=200
for epoch in range(1,epochs+1):
    train_loss=0
    s=0.
    for id_user in range(n_users):
        input=training_set[id_user][np.newaxis,:]
        target=input.clone()
        if torch.sum(target.data>0)>0:
            output=sae(input)
            target.require_grad=False #to make sure gradient is not computed wrt target, saves lot of computation, optimizes code
            output[target==0]=0 
            loss=criterion(output,target)
            mean_corrector=n_movies/float(torch.sum(target.data>0)+1e-10)
            loss.backward()
            train_loss+=np.sqrt(loss.data*mean_corrector)
            s+=1.
            optimizer.step()
    print("Epoch:"+str(epoch)+", train_loss:"+str(train_loss.numpy()/s))

Epoch:1, train_loss:1.0188570801299046
Epoch:2, train_loss:1.0184593483853062
Epoch:3, train_loss:1.0179813583228394
Epoch:4, train_loss:1.0175605846740787
Epoch:5, train_loss:1.0173636281730514
Epoch:6, train_loss:1.0168912691377254
Epoch:7, train_loss:1.016621627079467
Epoch:8, train_loss:1.0166316593692006
Epoch:9, train_loss:1.0163053834172853
Epoch:10, train_loss:1.016465964777472
Epoch:11, train_loss:1.0159745120294275
Epoch:12, train_loss:1.0160448027820121
Epoch:13, train_loss:1.0158894641022667
Epoch:14, train_loss:1.0157585912645812
Epoch:15, train_loss:1.015617168341563
Epoch:16, train_loss:1.0155244182048648
Epoch:17, train_loss:1.0153188533520017
Epoch:18, train_loss:1.0147980157037049
Epoch:19, train_loss:1.012555507688229
Epoch:20, train_loss:1.011510531444691
Epoch:21, train_loss:1.0088172823601538
Epoch:22, train_loss:1.0090561155802624
Epoch:23, train_loss:1.0051153027240192
Epoch:24, train_loss:1.004396343534597
Epoch:25, train_loss:1.0002776678900451
Epoch:26, train

In [38]:
a=np.array([1,2,4])
a[:,np.newaxis]

array([[1],
       [2],
       [4]])

In [83]:
test_loss=0
s=0.
for id_user in range(n_users):
    input=training_set[id_user][np.newaxis,:]
    target=test_set[id_user][np.newaxis,:]
    if torch.sum(target.data>0)>0:
        output=sae(input)
        target.requires_grad=False
        output[target==0]=0
        loss=criterion(output,target)
        mean_corrector=n_movies/float(torch.sum(target.data>0)+1e-10)
        test_loss+=np.sqrt(loss.data*mean_corrector)
        s+=1.
print("test_loss:"+str(test_loss.numpy()/s))

test_loss:0.9566399069393382
